In [244]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from fancyimpute import KNN
from sklearn.metrics import mean_squared_error
from math import sqrt
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import sys
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [245]:
df = pd.read_excel(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", sheet_name='Version1',engine='openpyxl',index_col='Date',parse_dates=True)
df.head()

,Money Market - Net liquidity injected,Money Market - Call Money,Money Market - Triparty Money,Money Market - Market Repo,Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot,Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward,Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward Cancel,...,NSDL FPI - D-PMO,NSDL FPI - D-ST,NSDL FPI - DVRR_SE,NSDL FPI - DVRR-PMO,NSDL FPI - DVRR-ST,NSDL FPI - H_SE,NSDL FPI - H-PMO,NSDL FPI - H-ST,NSDL FPI - Total,Value of Rupee
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-14,-2.915450,4.64,4.27,NaN,2297.0,861.0,488.0,47.0,89.0,36.0,...,-0.21,453.72,-0.21,41.77,41.56,0.65,0.0,0.65,588.68,71.45
2020-02-15,-3.093000,4.81,4.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.45
2020-02-16,-3.093130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.45
2020-02-17,-2.824517,4.93,4.85,4.92,1714.0,509.0,356.0,60.0,44.0,35.0,...,2.24,185.07,0.00,0.00,0.00,0.55,0.0,0.55,25.64,71.38
2020-02-18,-2.597337,4.95,4.89,4.88,3017.0,739.0,735.0,210.0,61.0,39.0,...,-0.61,-16.58,0.00,5.60,5.60,0.33,0.0,0.33,13.61,71.55


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1106 entries, 2020-02-14 to 2023-02-23
Data columns (total 71 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Money Market - Net liquidity injected                                     988 non-null    float64
 1   Money Market  - Call Money                                                808 non-null    float64
 2   Money Market - Triparty Money                                             808 non-null    float64
 3   Money Market - Market Repo                                                688 non-null    float64
 4   Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot           720 non-null    float64
 5   Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward         720 non-null    float64
 6   Foreign Exchange Turnover (Purchases) -Merchan

In [247]:
def fill_nan(i,df):
    df_copy = df[[i]]
    df_copy['interpolation'] = df_copy[i].interpolate(method = 'linear').fillna(df[i].mean())
    df_copy['forward_fill'] = df_copy[i].fillna(method='ffill').fillna(method='bfill')
    df_copy['backward_fill'] = df_copy[i].fillna(method='bfill').fillna(method='ffill')
    df_copy['Polynomial'] = df_copy[i].interpolate(method='polynomial', order=3).fillna(df[i].mean())

    train_size = int(len(df_copy) * 0.7)
    train, test = df_copy[0:train_size], df_copy[train_size:len(df_copy)]

    model_interp = ARIMA(train['interpolation'], order=(1,1,1))
    model_interp_fit = model_interp.fit()

    model_ffill = ARIMA(train['forward_fill'], order=(1,1,1))
    model_ffill_fit = model_ffill.fit()

    model_bfill = ARIMA(train['backward_fill'], order=(1,1,1))
    model_bfill_fit = model_bfill.fit()

    model_ply = ARIMA(train['Polynomial'], order=(1,1,1))
    model_ply_fit = model_ply.fit()

    predictions_interp = model_interp_fit.forecast(steps=len(test))
    predictions_ffill = model_ffill_fit.forecast(steps=len(test))
    predictions_bfill = model_bfill_fit.forecast(steps=len(test))
    predictions_ply = model_ply_fit.forecast(steps=len(test))

    rmse_interp = sqrt(mean_squared_error(test[i].fillna(0.0), predictions_interp))
    rmse_ffill = sqrt(mean_squared_error(test[i].fillna(0.0), predictions_ffill))
    rmse_bfill = sqrt(mean_squared_error(test[i].fillna(0.0), predictions_bfill))
    rmse_ply = sqrt(mean_squared_error(test[i].fillna(0.0), predictions_ply))

    minimum = sys.maxsize

    if rmse_bfill<minimum:
        method = "bfill - fflill"
        minimum = rmse_bfill
        min_df = df_copy['backward_fill'].tolist()

    if rmse_ffill<minimum:
        method = "ffill - bfill"
        minimum = rmse_ffill
        min_df = df_copy['forward_fill'].tolist()

    if rmse_ply<minimum:
        method = "Ploynomial Interpolation with mean"
        minimum = rmse_ply
        min_df = df_copy['Polynomial'].tolist()

    if rmse_interp<minimum:
        method = "Linear Interpolation with mean"
        minimum = rmse_interp
        min_df = df_copy['interpolation'].tolist()

    
    return method,minimum,min_df

In [248]:
daily = df.loc[:,"Money Market - Net liquidity injected":"Foreign Exchange Turnover (Sales) -Inter Bank FCY/FCY - Forward"]
columns = list(daily.columns)
daily2 = df.loc[:,"NSE Open Interest - Option Index Call Long":"NSDL FPI -  Total"]
columns2 = list(daily2.columns)

In [249]:
methods = {}
for i in columns:
    method,rmse,values =  fill_nan(i,df)
    print(f'Column {i} ---------> Method {method} -------> Rmse {rmse}')
    methods[i] = method
    df[i] = values

for i in columns2:
    method,rmse,values =  fill_nan(i,df)
    print(f'Column {i} ---------> Method {method} -------> Rmse {rmse}')
    methods[i] = method
    df[i] = values

Column Money Market - Net liquidity injected ---------> Method Ploynomial Interpolation with mean -------> Rmse 4.498787291689926
Column Money Market  - Call Money ---------> Method bfill - fflill -------> Rmse 2.4712308946589734
Column Money Market - Triparty Money ---------> Method Linear Interpolation with mean -------> Rmse 2.4740535151962026
Column Money Market - Market Repo ---------> Method ffill - bfill -------> Rmse 2.5677010663203967
Column Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot ---------> Method Ploynomial Interpolation with mean -------> Rmse 2800.747646999222
Column Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward ---------> Method Ploynomial Interpolation with mean -------> Rmse 1236.9100380206492
Column Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel ---------> Method Ploynomial Interpolation with mean -------> Rmse 841.9334285949727
Column Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot --

In [250]:
methods

{'Money Market - Net liquidity injected': 'Ploynomial Interpolation with mean',
 'Money Market  - Call Money': 'bfill - fflill',
 'Money Market - Triparty Money': 'Linear Interpolation with mean',
 'Money Market - Market Repo': 'ffill - bfill',
 'Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot': 'Ploynomial Interpolation with mean',
 'Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward': 'Ploynomial Interpolation with mean',
 'Foreign Exchange Turnover (Purchases) -Merchant FCY/INR - Forward Cancel': 'Ploynomial Interpolation with mean',
 'Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Spot': 'Ploynomial Interpolation with mean',
 'Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward': 'bfill - fflill',
 'Foreign Exchange Turnover (Purchases) -Merchant FCY/FCY - Forward Cancel': 'Ploynomial Interpolation with mean',
 'Foreign Exchange Turnover (Purchases) -Inter Bank FCY/INR - Spot': 'Ploynomial Interpolation with mean',
 'Foreign E

In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1106 entries, 2020-02-14 to 2023-02-23
Data columns (total 71 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Money Market - Net liquidity injected                                     1106 non-null   float64
 1   Money Market  - Call Money                                                1106 non-null   float64
 2   Money Market - Triparty Money                                             1106 non-null   float64
 3   Money Market - Market Repo                                                1106 non-null   float64
 4   Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot           1106 non-null   float64
 5   Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward         1106 non-null   float64
 6   Foreign Exchange Turnover (Purchases) -Merchan

In [252]:
monthly = df.loc[:,"NEER AND REER - Trade NEER":"External Commercial borrowings - Weighted Average Maturity"]
monthly = monthly.resample('MS').median()
columns3 = list(monthly.columns)
for i in columns3:
    method,rmse,values =  fill_nan(i,monthly,)
    print(f'Column {i} ---------> Method {method} -------> Rmse {rmse}')
    methods[i] = method
    monthly[i] = values
df_daily = monthly.resample('D').ffill()
df_daily = df_daily.loc['2020-02-14':'2023-02-23',:]
for i in list(df_daily.columns):
    values = df_daily[i].tolist()
    df[i] = values

Column NEER AND REER - Trade NEER ---------> Method bfill - fflill -------> Rmse 3.5356284570663004
Column NEER AND REER - Trade REER ---------> Method bfill - fflill -------> Rmse 4.877227447536594
Column NEER AND REER - Export NEER ---------> Method bfill - fflill -------> Rmse 3.3810755808514616
Column NEER AND REER - Export REER ---------> Method bfill - fflill -------> Rmse 4.652858902865005
Column Outward Remittances ---------> Method bfill - fflill -------> Rmse 2154.811415088703
Column Foreign Investment Inflows ---------> Method bfill - fflill -------> Rmse 3349.12198439295
Column Foreign Trade - Trade Balance ---------> Method bfill - fflill -------> Rmse 22837.445492437528
Column Foreign Trade - Trade Balance - Oil ---------> Method bfill - fflill -------> Rmse 9302.97469764889
Column Foreign Trade - Trade Balance - Non Oil ---------> Method bfill - fflill -------> Rmse 14607.824129020753
Column Maturity Breakdown - Up to One Month ---------> Method bfill - fflill -------> R

In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1106 entries, 2020-02-14 to 2023-02-23
Data columns (total 71 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Money Market - Net liquidity injected                                     1106 non-null   float64
 1   Money Market  - Call Money                                                1106 non-null   float64
 2   Money Market - Triparty Money                                             1106 non-null   float64
 3   Money Market - Market Repo                                                1106 non-null   float64
 4   Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot           1106 non-null   float64
 5   Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward         1106 non-null   float64
 6   Foreign Exchange Turnover (Purchases) -Merchan

In [254]:
weekly = df.loc[:,'FX_Reserves - Total Reserves':'FX_Reserves -Gold']
weekly_columns = list(weekly.columns)
for i in weekly_columns:
    method,rmse,values =  fill_nan(i,df)
    print(f'Column {i} ---------> Method {method} -------> Rmse {rmse}')
    methods[i] = method
    df[i] = values

Column FX_Reserves - Total Reserves ---------> Method Linear Interpolation with mean -------> Rmse 87.38161982172902
Column FX_Reserves -Foreign Currency ---------> Method Linear Interpolation with mean -------> Rmse 74.03700833045804
Column FX_Reserves -Gold ---------> Method Linear Interpolation with mean -------> Rmse 6.587552269692652


In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1106 entries, 2020-02-14 to 2023-02-23
Data columns (total 71 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Money Market - Net liquidity injected                                     1106 non-null   float64
 1   Money Market  - Call Money                                                1106 non-null   float64
 2   Money Market - Triparty Money                                             1106 non-null   float64
 3   Money Market - Market Repo                                                1106 non-null   float64
 4   Foreign Exchange Turnover (Purchases) - Merchant FCY/INR - Spot           1106 non-null   float64
 5   Foreign Exchange Turnover (Purchases) -Merchnat FCY/INR - Forward         1106 non-null   float64
 6   Foreign Exchange Turnover (Purchases) -Merchan

In [256]:
import pandas as pd
from openpyxl import load_workbook
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Version2'
df.index = df.index.strftime('%d-%m-%Y')
df.to_excel(writer, sheet_name=sheet_name, index=True)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")

In [257]:
df3 = pd.DataFrame.from_dict(methods,orient='index').reset_index()
df3.columns = ['Column','Type']
df3.set_index('Column')
book = load_workbook(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
writer = pd.ExcelWriter(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx", engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet_name = 'Report - Version2'
df3.to_excel(writer, sheet_name=sheet_name, index=True,startrow=3, startcol=0)
writer.save()
book.save(r"C:\Users\bhati\OneDrive\Desktop\Dataset_Fyp.xlsx")
df3

,Column,Type
0,Money Market - Net liquidity injected,Ploynomial Interpolation with mean
1,Money Market - Call Money,bfill - fflill
2,Money Market - Triparty Money,Linear Interpolation with mean
3,Money Market - Market Repo,ffill - bfill
4,Foreign Exchange Turnover (Purchases) - Mercha...,Ploynomial Interpolation with mean
...,...,...
65,External Commercial borrowings - Total Amount,bfill - fflill
66,External Commercial borrowings - Weighted Aver...,bfill - fflill
67,FX_Reserves - Total Reserves,Linear Interpolation with mean
68,FX_Reserves -Foreign Currency,Linear Interpolation with mean
